In [ ]:
#!pip install faker

In [ ]:
#pip install pymysql

In [31]:
#importing necessary libraries
import pandas as pd
import numpy as np
import mysql.connector
from datetime import datetime, timedelta

In [32]:
from faker import Faker
fake = Faker('en_IN')

In [33]:
#connection to sql database
import sqlalchemy, pymysql
from sqlalchemy.dialects import mysql
import pandas as pd
db = sqlalchemy.create_engine(
     sqlalchemy.engine.url.URL(
          drivername='mysql+pymysql',
          username='root',
          password="Yamini@kvt12",
          host='35.196.234.121',
          database="ufh-db"
     )
)

C:\Users\pothula.yamini\AppData\Local\Temp\ipykernel_7272\2382207158.py:5: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  sqlalchemy.engine.url.URL(


# Generate Customer Table Data

In [61]:
#column names of customer_master tables
cust_fields= ['customerid','name','address','city','state','pincode','update_timestamp']

In [62]:
#Dictionary of states and corresponding cities
states = {'Madhya Pradesh':['Indore','Bhopal','Gwalior','Ujjain','Ratlam'],
          'Maharashtra':['Mumbai','Nagpur','Pune'],
          'Andhra Pradesh' : ['Visakhpatnam','Vijaynagaram','Hyderabad','Rajahmundry'],
          'Karnataka':['Bangalore','Mangalore','Mysore'],
          'Uttar Pradesh':['Noida','Lucknow','Mathura'],
          'Jharkhand' : ['Ranchi'],
          'Chhattisgarh':['Raipur']}

In [63]:
#creating customer dataframe 
customer = pd.DataFrame(columns=cust_fields,index=range(1,1001))
Faker.seed(10)
for i in range(1,1001):
    customer['customerid'][i]=i
    customer['name'][i]=fake.name()
    customer['address'][i]=fake.street_address()    
    customer['state'][i]=fake.random_element(states.keys())
    customer['city'][i]=fake.random_element(states[customer['state'][i]])
    customer['pincode'][i]=fake.postcode()
    customer['update_timestamp'][i]=fake.date_time_this_year()

### loading customer table into  CUSTOMER_MASTER

In [66]:
#appending customer dataframe to CUSTOMER_MASTER table in sql instance
customer.to_sql('CUSTOMER_MASTER', con=db, if_exists='append',index=False)

1000

In [67]:
qry='select * from CUSTOMER_MASTER'

In [68]:
result= pd.read_sql(qry,con=db)

In [69]:
result.head()

,Customerid,Name,Address,City,State,Pincode,Update_timestamp
0,1,Aaryahi Khurana,"774, Badal Street",Lucknow,Uttar Pradesh,135062,2022-08-23 06:08:38
1,2,Vaibhav Kade,47/245\nKari Street,Raipur,Chhattisgarh,960903,2022-02-22 00:42:52
2,3,Romil Loyal,587\nKeer Marg,Ratlam,Madhya Pradesh,823630,2022-01-13 07:54:02
3,4,Romil Taneja,"12/69, Rama Path",Mumbai,Maharashtra,723566,2022-06-23 01:21:38
4,5,Pihu Devan,"82/14, Dora Chowk",Pune,Maharashtra,275563,2022-01-02 22:40:59


# Generate Product_master Table Data

In [70]:
product_attributes = ['Productid','Productcode','Productname','Sku','Rate','Isactive']

In [71]:
# List of available products on UFH
products = ['Organic India Chyavanprash',
            'Aashirwad Atta',
            'Herbal Strategi Oil',
            'Herbal Strategi Floor Cleaner',
            'Soulfull Ragi Bites Choco Fills',
            'Natureland Puffed Wheat',
            'Natureland Pasta Macroni',
            'Natureland Rice Poha',
            'Pure Sure Organic Chakli',
            'Organic Wellness Chilli Chocolate',
            'Natureland Green Tea',
            'Organic India Tulsi Tummy',
            'Organic Wellness Turmeric Chocolate',
            'Organic Wellness Ginger Tea',
            '24 Mantra Organic Orange Juice',
            '24 Mantra Organic Mango Juice',
            'Ayuda Organics Desi Jaggery',
            '24 Mantra Handpounded Rice',
            '24 Mantra Idly Rice',
            'Natureland Brown Sugar',
            'Natureland Sunflower Oil',
            'Natureland Mustard Oil',
            'Natureland Jaggery Powder',
            'Natureland Red Rice',
            'Natureland Rice Flour',
            'Biotique Bio White Face Wash',
            'Vitro Naturals Aloe Face Wipes',
            'Biotique Bio Honey Gel Face Wash',
            'Biotique Bio Orris Root',
            'Vitro Naturals Aloe Face Wash',
            'BioPure Hand Sanitizer',
            'Soultree Lipgloss Nude Pink',
            'Soultree Lipgloss Rich Earth',
            'Soultree Lipgloss Lush Berry',
            'Soultree Lipstick Iced Plum',
            'Soultree Kajal Pure Black',
            'Soultree Grey Glow Kajal',
            'Ayuda Organics Chhundo (Sun Dried)',
            'Ayuda Organics Raw Honey',
            'Ayuda Organics Homemade Mango Pickle',
            'Ayuda Organics Homemade Chana Methi Pickle',
            'Natureland Garlic Pickle',
            'Natureland Pineapple Jam',
            'Natureland Pineapple Jam',
            'Natureland Honey',
            'Natureland Walnut',
            'Natureland Cashew',
            'Natureland Raisins',
            'Natureland Almonds',
            'Pure Sure Organic Rice Dosa Mix'
           ]

In [72]:
#creating product dataframe
product = pd.DataFrame(columns = product_attributes, index=range(1,201))
Faker.seed(10)
for i in range(1,51):
    product_name = products[i-1]
    for j in range((i*4)-3,(i*4)+1):        
        product['Productid'][j]=j
        product['Productcode'][j]=fake.bothify('????-########', letters='ABCDE')
        product['Productname'][j]= product_name
        sku = 0
        if j%4 == 1:
            sku = fake.random_int(min=1, max=10)
        else:
            sku = int(product['Sku'][j-1][:-2]) + 2
        product['Sku'][j] = str(sku) + 'KG'
        product['Rate'][j]=round(np.random.normal(1000,200))
        product['Isactive'][j]=fake.boolean()

### loading PRODUCT_MASTER table

In [73]:
product.to_sql('PRODUCT_MASTER', con=db, if_exists='append',index=False)

200

In [74]:
qry='select * from PRODUCT_MASTER'

In [75]:
result= pd.read_sql(qry,con=db)

In [76]:
result.head(20)

,Productid,Productcode,Productname,Sku,Rate,Isactive
0,1,DCBA-90679037,Organic India Chyavanprash,9KG,965,0
1,2,CDDC-51350629,Organic India Chyavanprash,11KG,1003,0
2,3,DEDA-47245273,Organic India Chyavanprash,13KG,913,0
3,4,CEDD-03234853,Organic India Chyavanprash,15KG,920,0
4,5,AADC-19582363,Aashirwad Atta,10KG,895,0
5,6,BAAD-18126959,Aashirwad Atta,12KG,1147,1
6,7,EEBA-35667342,Aashirwad Atta,14KG,1040,1
7,8,DBAE-74222755,Aashirwad Atta,16KG,782,0
8,9,EBDD-05531476,Herbal Strategi Oil,4KG,850,0
9,10,DDDB-84879712,Herbal Strategi Oil,6KG,1147,0


# Generate Order details table data

In [77]:
#creating orders_recieved dataframe
order_detail_fields = ['orderid', 'customerid', 'order_status_update_timestamp', 'order_status']
orders_Received = pd.DataFrame(columns=order_detail_fields, index=range(1, 60001))

x=1
j=1
for i in range(1, 20001):
    f=fake.random_element(customer['customerid'])
    for k in range (j,j+3):
        orders_Received['orderid'][k] = x
        orders_Received['customerid'][k]=f

    orders_Received['order_status_update_timestamp'][j] = fake.date_time_this_year()
    orders_Received['order_status'][j] = 'Received'
    orders_Received['order_status_update_timestamp'][j+1] = orders_Received['order_status_update_timestamp'][j] + timedelta(seconds=fake.random_int(min=1, max=86400))
    orders_Received['order_status'][j+1] = 'In_progress'
    orders_Received['order_status_update_timestamp'][j+2] = orders_Received['order_status_update_timestamp'][j+1] + timedelta(seconds=fake.random_int(min=1, max=172800))
    orders_Received['order_status'][j+2] = 'Delivered'
    j+=3
    x+=1

### loading order details table

In [78]:
orders_Received.to_sql('ORDER_DETAILS', con=db, if_exists='append',index=False)

60000

In [79]:
qry='select * from ORDER_DETAILS'

In [80]:
result= pd.read_sql(qry,con=db)

In [81]:
result.head()

,Orderid,Customerid,Order_status_update_timestamp,Order_status
0,1,435,2022-11-12 13:21:02,Delivered
1,1,435,2022-11-11 04:05:53,In_progress
2,1,435,2022-11-10 18:31:13,Received
3,2,71,2022-08-19 07:09:17,Delivered
4,2,71,2022-08-17 19:55:29,In_progress


# Generate Order items table data 

In [82]:
order_item_fields = ['orderid','productid','quantity']

In [83]:
order_items = pd.DataFrame(columns=order_item_fields, index = range(1,50001))

In [84]:
for i in range(1,50001):
    order_items['orderid'][i]=fake.random_element(orders_Received['orderid'])
    order_items['productid'][i]=fake.random_element(product['Productid'])
    order_items['quantity'][i]=fake.random_int(min=1, max=20)

In [85]:
order_items = order_items.drop_duplicates(subset=['orderid', 'productid'],keep='first')

### Loading ORDER_ITEMS table

In [86]:
order_items.to_sql('ORDER_ITEMS', con=db, if_exists='append',index=False)

49706

In [87]:
qry='select * from ORDER_ITEMS'

In [88]:
result= pd.read_sql(qry,con=db)

In [89]:
result.head()

,Orderid,Productid,Quantity
0,1,71,14
1,1,170,20
2,2,12,10
3,3,52,9
4,4,43,13
